In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split  
from sklearn import preprocessing
import optuna 
import lightgbm as lgb
from sklearn.metrics import accuracy_score, roc_auc_score,precision_score, confusion_matrix,recall_score
from sklearn.model_selection import cross_val_score, train_test_split
import os 

In [2]:
train_data=pd.read_csv('/home/iai/son/lastcheck/spambase_train_data.csv')
test_data=pd.read_csv('/home/iai/son/lastcheck/spambase_test_data.csv')

In [3]:
train_stage_features=['0.7', '278', '61', '0', '0.34', '0.30', '0.5', '0.44', '0.40', '0.778',
                        '0.24', '1.93', '0.39', '1.29', '0.43', '0.2', '0.36', '0.17', '0.20', 
                        '0.28', '0.21', '0.32.1', '0.41', '0.29', '3.756', '0.35', '0.25', '0.15', 
                        '0.64.1', '0.38', '0.45', '0.6', '0.33', '0.42', '0.14', '0.9', '0.10', 
                        '0.3', '0.13', '0.8', '0.19', '0.23', '0.18', '0.22', '0.16', '0.64.2', 
                        '0.1', '0.31', '0.27', '0.11', '0.4', '0.12', '0.26', '0.64', '0.37', '0.32', '0.96']
test_stage_features=['0.7', '278', '61', '0', '0.34', '0.30', '0.5', '0.44', '0.40', '0.778',
                        '0.24', '1.93', '0.39', '1.29', '0.43', '0.2', '0.36', '0.17', '0.20']

In [4]:
train_X=train_data.loc[:,train_stage_features]
train_y=train_data['1']
new_test_data_X=test_data.loc[:,test_stage_features]
new_test_data_y=test_data['1']

In [5]:
ratio=train_y.value_counts()[0]/train_y.value_counts()[1]

In [6]:
def objective(trial):
    # Define hyperparameters to optimize 
    params={
        'boosting_type':'gbdt',
        'objective':'binary',
        'metric':'binary_logloss',
        'learning_rate':trial.suggest_loguniform('learning_rate',0.01,0.1),
        'max_depth':trial.suggest_int('max_depth',3,9),
        'n_estimators':trial.suggest_int("n_estimators",100,1000)
    }
    # Train and evaluate model 
    lgb_cv=lgb.LGBMClassifier(**params, random_state=42,scale_pos_weight=ratio)
    scores=cross_val_score(lgb_cv,train_X,train_y,cv=5,scoring='roc_auc')
    auc=scores.mean()
    return auc 

# Define study object and optimize 

study=optuna.create_study(direction='maximize',study_name='lgb_boost_opt',load_if_exists=True)
study.optimize(objective, n_trials=50)

# Print best hyperparameters and auc
print(f'Best hyperparameters: {study.best_params}')
print(f'Best AUC: {study.best_value:.4f}')

[I 2023-04-26 16:03:27,053] A new study created in memory with name: lgb_boost_opt
/tmp/ipykernel_1366309/2141867322.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate':trial.suggest_loguniform('learning_rate',0.01,0.1),
[I 2023-04-26 16:03:30,585] Trial 0 finished with value: 0.9901069525233599 and parameters: {'learning_rate': 0.027578140510599508, 'max_depth': 7, 'n_estimators': 995}. Best is trial 0 with value: 0.9901069525233599.
/tmp/ipykernel_1366309/2141867322.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate':trial.suggest_loguniform('learning_rate',0.01,0.1),
[I 2023-04-26 16:03:31,035] Trial 1 finishe

Best hyperparameters: {'learning_rate': 0.04515243459735279, 'max_depth': 5, 'n_estimators': 578}
Best AUC: 0.9908


In [7]:
lgb=lgb.LGBMClassifier(learning_rate= 0.04515243459735279,max_depth= 5,n_estimators=578,scale_pos_weight=ratio,random_state=42)

In [8]:
lgb.fit(train_X,train_y)

LGBMClassifier(learning_rate=0.04515243459735279, max_depth=5, n_estimators=578,
               random_state=42, scale_pos_weight=1.5879043600562588)

In [9]:
train_prob=pd.DataFrame()
train_prob['prob']=lgb.predict_proba(train_X)[:,1]

In [10]:
train_prob['1_prob']=1-train_prob['prob']
train_prob['odds']=train_prob['prob']/train_prob['1_prob']

In [11]:
train_prob['loogits']=np.log(train_prob['odds'])
train_X_new=train_X[test_stage_features]

In [12]:
import optuna
from lightgbm import LGBMRegressor

# Objective 함수 정의
def objective(trial):
    
    # 하이퍼파라미터 탐색할 공간 정의
    params = {
        'n_estimators': trial.suggest_int('n_estimators',100,1000),
        'max_depth': trial.suggest_int('max_depth',3,9),
        'learning_rate': trial.suggest_loguniform('learning_rate',0.01,0.1),
    }
    
    # LGBMRegressor 모델 객체 생성
    model = LGBMRegressor(**params, random_state=42)
    
    # 교차검증 수행하여 모델 성능 측정
    scores = -1 * cross_val_score(model, train_X_new, train_prob['loogits'],
                                  cv=5, scoring='neg_mean_squared_error')
    
    # 교차검증 평균 점수 리턴
    return np.mean(scores)


# Optuna study 생성
study = optuna.create_study(direction='minimize')

# study 실행 (n_trials는 시도 횟수)
study.optimize(objective, n_trials=50)

# 최적화된 하이퍼파라미터 값 출력
print(study.best_params)

[I 2023-04-26 16:06:07,175] A new study created in memory with name: no-name-c8fdcc4a-3ff2-4d69-ab2f-63a61c6e7a40
/tmp/ipykernel_1366309/4075368247.py:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate': trial.suggest_loguniform('learning_rate',0.01,0.1),
[I 2023-04-26 16:06:07,707] Trial 0 finished with value: 8.216450031806591 and parameters: {'n_estimators': 347, 'max_depth': 5, 'learning_rate': 0.04424599490223671}. Best is trial 0 with value: 8.216450031806591.
/tmp/ipykernel_1366309/4075368247.py:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate': trial.suggest_loguniform('learning_rate',0.01,0.1),
[I 2023-04-

{'n_estimators': 581, 'max_depth': 8, 'learning_rate': 0.04151307819139772}


In [13]:
# Print best hyperparameters and auc

print(f'Best hyperparameters: {study.best_params}')
print(f'Best RMSE: {study.best_value:.4f}')

Best hyperparameters: {'n_estimators': 581, 'max_depth': 8, 'learning_rate': 0.04151307819139772}
Best RMSE: 7.8316


In [14]:
model = LGBMRegressor(n_estimators=581,max_depth=8,learning_rate=0.04151307819139772)

In [15]:
model.fit(train_X_new,train_prob['loogits'])

LGBMRegressor(learning_rate=0.04151307819139772, max_depth=8, n_estimators=581)

In [16]:
test_data=pd.DataFrame()
test_data['loogits']=model.predict(new_test_data_X)

In [17]:
test_data['new_prob']=1/(1+np.exp(-test_data['loogits']))

In [18]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score

def get_clf_prob(y_test, probability):
  pred=np.where(probability > 0.5,1,0)
  confusion=confusion_matrix(y_test, pred)
  accuracy=accuracy_score(y_test,pred)
  precision=precision_score(y_test,pred) 
  recall=recall_score(y_test,pred) 
  # F1 스코어 추가 
  f1=f1_score(y_test,pred,average='macro')
  Roc_score=roc_auc_score(y_test,probability)
  print('임계값: ', 0.5) 
  print('오차행렬')
  print(confusion) 
  # f1 score print 추가 
  print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f}, F1:{3:.4f}, AUC:{4: .4f}'.format(accuracy,precision,recall,f1,Roc_score))

In [19]:
get_clf_prob(new_test_data_y,test_data['new_prob'])

임계값:  0.5
오차행렬
[[502  28]
 [ 73 317]]
정확도: 0.8902, 정밀도: 0.9188, 재현율: 0.8128, F1:0.8856, AUC: 0.9515
